## 모델 불러오기

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
# CTCLayer 클래스 정의
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        return y_pred

In [4]:
# 모델 로드
with keras.utils.CustomObjectScope({'CTCLayer': CTCLayer}):
    model_path = 'CAPTCHA_CRNN_model(Dataset_C,batch_size=32,normalized,lr=0.001).h5'
    model = keras.models.load_model(model_path)

2023-12-07 18:18:10.190139: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-12-07 18:18:10.190161: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-07 18:18:10.190167: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-07 18:18:10.190201: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-07 18:18:10.190217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
# 데이터셋 경로 및 기타 파라미터 설정
from pathlib import Path

dataset_path = Path('5char_test')
img_height = 50
img_width = 200
batch_size = 32

# 이미지와 레이블 리스트 로드
images = sorted(list(map(str, dataset_path.glob("*.png"))))
labels = [img.split(os.path.sep)[-1].split(".png")[0] for img in images]

# 레이블에서 고유한 문자들 추출 및 매핑
characters = set(char for label in labels for char in label)
characters = sorted(list(characters))

char_to_num = layers.StringLookup(vocabulary=list(characters), mask_token=None)
num_to_char = layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)

max_length = max([len(label) for label in labels])

In [24]:
# 이미지 전처리 함수 정의
def encode_single_sample(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.transpose(img, perm=[1, 0, 2])
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    padding_size = max_length - tf.shape(label)[0]
    padding = tf.zeros(padding_size, dtype=label.dtype)
    label = tf.concat([label, padding], axis=0)
    return {"image": img, "label": label}

# TensorFlow 데이터셋 생성 및 전처리
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(encode_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(batch_size).shuffle(buffer_size=len(images))

In [25]:
# 모델 성능 평가
results = model.evaluate(dataset)

# 결과 출력
print(f"Loss: {results}")

1/1 [==============================] - 0s 103ms/step - loss: 67.7909
Loss: 67.7908706665039


In [26]:
# 모델 예측
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
prediction_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 200, 50, 1)]      0         
                                                                 
 Conv1 (Conv2D)              (None, 200, 50, 32)       320       
                                                                 
 batch_normalization_4 (Bat  (None, 200, 50, 32)       128       
 chNormalization)                                                
                                                                 
 pool1 (MaxPooling2D)        (None, 100, 25, 32)       0         
                                                                 
 Conv2 (Conv2D)              (None, 100, 25, 64)       18496     
                                                                 
 batch_normalization_5 (Bat  (None, 100, 25, 64)       256       
 chNormalization)                                          

In [27]:
# 예측 함수 정의
def predict(model, batch):
    prediction = model.predict(batch)
    # 모델의 출력값을 문자열로 변환
    output = tf.keras.backend.ctc_decode(
        prediction, input_length=np.ones(prediction.shape[0]) * prediction.shape[1], greedy=True
    )[0][0]
    # 변환된 문자열을 숫자로 변환
    out = tf.keras.backend.get_value(output)
    # 숫자를 문자로 변환
    result = []
    for i in out:
        result.append(num_to_char(i))
    return result

In [28]:
# 예측 결과 출력
for batch in dataset.take(1):
    batch_images = batch["image"]
    batch_labels = batch["label"]

    preds = predict(prediction_model, batch_images)

    print("Labels: ")
    for label in batch_labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        print(label)

    print("Predictions: ")
    for pred in preds:
        print(pred)

1/1 [==============================] - 1s 529ms/step
Labels: 
2kh0i
3othj
cdplh
cktva
dqdpa
ffwln
k0osr
n94vb
oyixx
plncr
pvixh
r89ll
r8h2m
s0otr
vdxyn
Predictions: 
tf.Tensor(
[b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]'], shape=(50,), dtype=string)
tf.Tensor(
[b'3' b'[UNK]' b'[UNK]' b'k' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UN